In [49]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [50]:
#Step0:Dataset

#load a dataset from sklearn.datasets
dataset = datasets.load_breast_cancer()

#extract the data and target
data, target = dataset.data, dataset.target

#reshape the target to uniform datastructure
target = np.resize(target, (data.shape[0],1)) #data shape is (569, 30), target shape should be (569, 1)

#spilt the dataset into training set and test set(Note:data and targets should be spilt at the same time)
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 1)

#rescale the data(Note: do not rescale the target)
rescale = StandardScaler()
train_data = rescale.fit_transform(train_data)
test_data = rescale.fit_transform(test_data)

#change the datastructure from numpy to torch for model evaluation
train_data = torch.from_numpy(train_data.astype(np.float32))
train_target = torch.from_numpy(train_target.astype(np.float32))
test_data = torch.from_numpy(test_data.astype(np.float32))
test_target = torch.from_numpy(test_target.astype(np.float32))

In [51]:
#Step1:Model

class LogisticRegression(nn.Module):
    def __init__(self, input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_features, 1) #binary classification
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegression(data.shape[1])

In [52]:
#Step3:Loss and optimizer
learning_rate = 0.001
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [54]:
#Step4:Training loop
num_epoch = 100
for epoch in range (num_epoch):
    #foward pass
    target_predicted = model(train_data)
    #calculate loss
    loss = loss_function(target_predicted, train_target)
    #backward pass, calculate the gradients
    loss.backward()
    #update weights and bias
    optimizer.step()
    #remember to reset the gradients
    optimizer.zero_grad()
    #print some information when training
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss: {loss.item():.3f}')

#test the training result(Note: use no_grad computation, we do not want to accumulate gradients when testing)
with torch.no_grad():
    Prediction = model(test_data)
    #binarize output, because we want to see logistic result
    Prediction = Prediction.round()
    accuracy = Prediction.eq(test_target).sum() / float(test_target.shape[0])
    print(f'test accuracy:{accuracy:.3f}')


epoch: 10, loss: 0.527
epoch: 20, loss: 0.477
epoch: 30, loss: 0.436
epoch: 40, loss: 0.402
epoch: 50, loss: 0.374
epoch: 60, loss: 0.350
epoch: 70, loss: 0.330
epoch: 80, loss: 0.312
epoch: 90, loss: 0.297
epoch: 100, loss: 0.283
test accuracy:0.939
